In [5]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import skimage
from skimage import transform
import matplotlib as mpl
import matplotlib.pyplot as plt
import os,glob

import random

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
class COVNet(keras.Model):
    def __init__(self, n_classes):
        super(COVNet,self).__init__()
        self.basemodel = keras.applications.ResNet50V2(input_shape=(256,256,3), 
                                                  include_top=False,
                                                  weights='imagenet')
        self.pooling_layer = layers.GlobalAveragePooling2D()
        self.classifer = layers.Dense(n_classes)
        self.n_classes = n_classes
        
    def call(self, x ,training = True):#  ( b,slices,x,y,c)
        x = tf.squeeze(x,axis = 0) #(slices,x,y,c)
        features = self.basemodel(x,training) #( slices,x,y,2048)
        pooled_features = self.pooling_layer(features) # (slices,2048*x*y)
        flattened_features = tf.reduce_max(pooled_features,axis = 0, keepdims=True) #(1,2048*x*y) max_pooling
        logit = self.classifer(flattened_features)
        return logit

In [4]:
# net = COVNet(2)
# x = tf.random.normal([1,30,256,256,3])
# y = tf.convert_to_tensor([1.],tf.float32)
# loss = tf.keras.losses.CategoricalCrossentropy()
# optim = keras.optimizers.Adam()
# with tf.GradientTape() as tape:
#     logit = net(x)
#     losses = loss(y,logit)
# grads = tape.gradient(losses,net.trainable_variables)
# optim.apply_gradients(zip(grads,net.trainable_variables))

In [8]:
dir_0 = glob.glob('E:\set\\0\*')[:100]
dir_1 = glob.glob('E:\set\\1\*')
filedirs = dir_0+dir_1

In [9]:
filedirs[0].split('\\')[-2]

'0'

In [11]:
filedirs[-1].split('\\')[-2]

'1'

In [ ]:
# filedirs = glob.glob('E:\data\CT\平扫肺窗\*\*')
# datainfo = pd.read_csv('E:\\data\\影像+临床资料-修订.csv',encoding='gb18030')
# heav_patient = list(datainfo.iloc[:,[2,5]].dropna().iloc[:,0].values)


#保存的文件路径是没有顺序的，需要用读取器读取，就可以按顺序
def get_series_and_processing(img_path):
    reader = sitk.ImageSeriesReader()
    img_names = reader.GetGDCMSeriesFileNames(img_path)
    reader.SetFileNames(img_names)
    image3D = reader.Execute()
    
    dcm = sitk.IntensityWindowing(image3D,    windowMinimum=-1000,    windowMaximum=0)
    arr = sitk.GetArrayFromImage(dcm)
    
    arr = arr[::-1]
    arr = arr[6:30]
    arr = np.transpose(transform.resize(np.transpose(arr,[1,2,0]),(256,256)),[2,0,1] )# x, y, z
    arr = np.expand_dims(arr,axis=0)
    
    return arr

net = COVNet(2)
criteon = tf.keras.losses.CategoricalCrossentropy()
optim = keras.optimizers.Adam(learning_rate=0.01)

epochs = 100

for epoch in range(epochs):
    
    losses = 0
    
    acc_count = 0
    
    i = 0
    
    print()
    print('Computing (epoch={0}/{1})'.format(epoch+1,epochs),end=' ')
    
    random.shuffle(filedirs)
    
    for filename in filedirs:
        os.chdir(filename)
        x = tf.cast(tf.stack([tf.convert_to_tensor(get_series_and_processing('.'))]*3,axis=-1),tf.float32)
#         y = tf.cast(tf.convert_to_tensor(np.array([int(filename.split('\\')[-2] in heav_patient)])),tf.float32)
        y = tf.cast(tf.convert_to_tensor(np.array([int(filename.split('\\')[-2])])),tf.float32)
        with tf.GradientTape() as tape:
            logit = net(x)
            loss = criteon(y,logit)
            
        pred = tf.cast(tf.argmax(tf.math.softmax(logit),axis=1),tf.float32)
        losses += loss.numpy()
        acc_count += int(tf.equal(pred,y).numpy())
        grad = tape.gradient(loss,net.trainable_variables)
        optim.apply_gradients(zip(grad,net.trainable_variables))
        
        i+=1
        if i%30 == 0:
            print('.',end='')
    print()

    avg_loss = float(losses/len(filedirs))
    acc = float(acc_count/len(filedirs))
    print('epoch',epoch+1,' loss:',avg_loss,' acc:',acc)
    


Computing (epoch=1/100) ......
epoch 1  loss: 7.422182208076518  acc: 0.47058823529411764

Computing (epoch=2/100) ......
epoch 2  loss: 7.4987930461047165  acc: 0.46524064171123

Computing (epoch=3/100) ......
epoch 3  loss: 7.4987930461047165  acc: 0.46524064171123

Computing (epoch=4/100) ......
epoch 4  loss: 7.4987930461047165  acc: 0.46524064171123

Computing (epoch=5/100) ......
epoch 5  loss: 7.4987930461047165  acc: 0.46524064171123

Computing (epoch=6/100) ......
epoch 6  loss: 7.4987930461047165  acc: 0.46524064171123

Computing (epoch=7/100) ....

In [ ]:
# filedirs = glob.glob('E:\data\CT\平扫肺窗\*\*')
# datainfo = pd.read_csv('E:\\data\\影像+临床资料-修订.csv',encoding='gb18030')
# heav_patient = list(datainfo.iloc[:,[2,5]].dropna().iloc[:,0].values)


#保存的文件路径是没有顺序的，需要用读取器读取，就可以按顺序
def get_series_and_processing(img_path):
    reader = sitk.ImageSeriesReader()
    img_names = reader.GetGDCMSeriesFileNames(img_path)
    reader.SetFileNames(img_names)
    image3D = reader.Execute()
    
    dcm = sitk.IntensityWindowing(image3D,    windowMinimum=-1000,    windowMaximum=0)
    arr = sitk.GetArrayFromImage(dcm)
    
    arr = arr[::-1]
    arr = arr[6:30]
    arr = np.transpose(transform.resize(np.transpose(arr,[1,2,0]),(256,256)),[2,0,1] )# x, y, z
    arr = np.expand_dims(arr,axis=0)
    
    return arr

net = COVNet(2)
criteon = tf.keras.losses.CategoricalCrossentropy()
optim = keras.optimizers.Adam(learning_rate=0.001)

epochs = 100

for epoch in range(epochs):
    
    losses = 0
    
    acc_count = 0
    
    i = 0
    
    print()
    print('Computing (epoch={0}/{1})'.format(epoch+1,epochs),end=' ')
    
    random.shuffle(filedirs)
    
    for filename in filedirs:
        os.chdir(filename)
        x = tf.cast(tf.stack([tf.convert_to_tensor(get_series_and_processing('.'))]*3,axis=-1),tf.float32)
#         y = tf.cast(tf.convert_to_tensor(np.array([int(filename.split('\\')[-2] in heav_patient)])),tf.float32)
        y = tf.cast(tf.convert_to_tensor(np.array([int(filename.split('\\')[-2])])),tf.float32)
        with tf.GradientTape() as tape:
            logit = net(x)
            loss = criteon(y,logit)
            
        pred = tf.cast(tf.argmax(tf.math.softmax(logit),axis=1),tf.float32)
        losses += loss.numpy()
        acc_count += int(tf.equal(pred,y).numpy())
        grad = tape.gradient(loss,net.trainable_variables)
        optim.apply_gradients(zip(grad,net.trainable_variables))
        
        i+=1
        if i%30 == 0:
            print('.',end='')
    print()

    avg_loss = float(losses/len(filedirs))
    acc = float(acc_count/len(filedirs))
    print('epoch',epoch+1,' loss:',avg_loss,' acc:',acc)
    


Computing (epoch=1/100) ......
epoch 1  loss: 7.421410668979991  acc: 0.5240641711229946

Computing (epoch=2/100) ......
epoch 2  loss: 7.4987930461047165  acc: 0.5347593582887701

Computing (epoch=3/100) ......
epoch 3  loss: 7.4987930461047165  acc: 0.5347593582887701

Computing (epoch=4/100) ......
epoch 4  loss: 7.4987930461047165  acc: 0.5347593582887701

Computing (epoch=5/100) ......
epoch 5  loss: 7.4987930461047165  acc: 0.5347593582887701

Computing (epoch=6/100) ......
epoch 6  loss: 7.4987930461047165  acc: 0.5347593582887701

Computing (epoch=7/100) ......
epoch 7  loss: 7.4987930461047165  acc: 0.5347593582887701

Computing (epoch=8/100) 